In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# Topic: EX2 - Turbofan RUL Prediction
**Task**: Predict the remaining useful life (RUL) of turbofan engines based on given sensor data (time series data). It is a forcasting problem, where the goal is to predict the number of cycles an engine will last before it fails.
**Data**: Turbofan engine degradation simulation data (NASA) - [Link](https://data.nasa.gov/dataset/Turbofan-Engine-Degradation-Simulation-Data-Set/vrks-gjie). See also in the topic [introduction notebook](https://github.com/nina-prog/damage-propagation-modeling/blob/2fb8c1a1102a48d7abbf04e4031807790a913a99/notebooks/Turbofan%20remaining%20useful%20life%20Prediction.ipynb).

**Subtasks**:
1. Perform a deep **exploratory data analysis (EDA)** on the given data.
2. Implement a more efficient **sliding window method** for time series data analysis. -> 🎯 **Focus on this task**
3. Apply **traditional machine learning methods** (SOTA) to predict the remaining useful life. Includes data preparation, feature extraction, feature selection, model selection, and model parameter optimization.
4. Create **neural network models** to predict the remaining useful life. Includes different architectures like Convolutional Neural Networks (CNN), Recurrent Neural Networks (RNN), or Attention Models. Note: You can search for SOTA research papers and reproduce current state-of-the-art models.


# Imports + Settings

In [3]:
# third-party libraries
import pandas as pd
import numpy as np
import os
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint

import time

import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
# source code
from src.utils import load_data, load_config, train_val_split_by_group
from src.nn_utils import scale_data, create_sliding_window
from src.nn_util.nn_models.ligthning.cnnModel2 import CNNModel2 as CNNModel
from src.nn_util.nn_models.ligthning.cnnModel1 import CNNModel1 as CNNModel1
from src.nn_util.nn_models.ligthning.exampleLSTMModel import ExampleLSTMModel as LSTMModel
from src.nn_util.datamodule.lightning.turbofanDatamodule import TurbofanDatamodule
from src.data_cleaning import clean_data

In [5]:
# settings
sns.set_style("whitegrid")
sns.set_palette("Set2")
sns.set(rc={"figure.dpi":100, 'savefig.dpi':200})
sns.set_context('notebook')

In [6]:
np.random.seed(42)

# Paths

In [7]:
# Make sure to execute this cell only once for one kernel session, before running any other cell below.
os.chdir("../") # set working directory to root of project
os.getcwd() # check current working directory

'C:\\Users\\Johannes\\PycharmProjects\\damage-propagation-modeling'

In [8]:
PATH_TO_CONFIG = "configs/config.yaml"

# Load Config + Data

In [9]:
config = load_config(PATH_TO_CONFIG) # config is dict

In [61]:
train_data, test_data, test_RUL_data = load_data(config_path=PATH_TO_CONFIG, dataset_num=1)

2024-05-30 17:34:54 [src.utils:60] [INFO] >>>> Loading data set 1...
2024-05-30 17:34:54 [src.utils:89] [INFO] >>>> Loaded raw data for dataset 1.
2024-05-30 17:34:54 [src.utils:90] [INFO] >>>> Train Data: (20631, 26)
2024-05-30 17:34:54 [src.utils:91] [INFO] >>>> Test Data: (13096, 26)
2024-05-30 17:34:54 [src.utils:92] [INFO] >>>> Test RUL Data: (100, 1)


# 📍 << Subtask X: TOPIC >>

[TEMPLATE]

Findings:
* Interpretation of plots
* or other key take aways from previous code

In [37]:
# [TEMPLATE] - save processed data (as pickle)
df = pd.DataFrame()
timestamp = time.strftime("%Y%m%d-%H%M%S")
df.to_pickle(f"{config['paths']['processed_data_dir']}ex2_topic_{timestamp}.pkl")

In [38]:
# [TEMPLATE] - save data predictions (as csv)
df = pd.DataFrame()
timestamp = time.strftime("%Y%m%d-%H%M%S")
df.to_csv(f"{config['paths']['prediction_dir']}ex2_topic_{timestamp}.csv", sep=',', decimal='.')

In [13]:
# [TEMPLATE] - save plot results (as png)
fig = plt.figure(figsize=(9, 6))
timestamp = time.strftime("%Y%m%d-%H%M%S")
fig.savefig(f"{config['paths']['plot_dir']}ex2_topic_{timestamp}.png")

<Figure size 900x600 with 0 Axes>

In [39]:
window_size = 30

In [40]:
# Setting the seed
pl.seed_everything(21)

Seed set to 21


21

In [41]:
#scoring: Dict[str, make_scorer] = {
#    'mae': make_scorer(mean_absolute_error),
#    'mse': make_scorer(mean_squared_error),
#    'r2': make_scorer(r2_score)
#}

In [62]:
cleaned_train, cleaned_test = clean_data(train_data, test_data, method=None, ignore_columns=['UnitNumber', 'Cycle'], threshold_missing=0.1, threshold_corr=0.0, contamination=0.05)

2024-05-30 17:34:58 [src.data_cleaning:134] [INFO] >>>> Cleaning train and test data...
2024-05-30 17:34:58 [src.data_cleaning:136] [INFO] >>>> Formatting column types...
2024-05-30 17:34:58 [src.data_cleaning:69] [DEBUG] >>>> Found 0 categorical columns: []
2024-05-30 17:34:58 [src.data_cleaning:69] [DEBUG] >>>> Found 0 categorical columns: []
2024-05-30 17:34:58 [src.data_cleaning:141] [INFO] >>>> Handling duplicates...
2024-05-30 17:34:58 [src.data_cleaning:146] [INFO] >>>> Removing outliers...
2024-05-30 17:34:58 [src.outlier_detection:150] [DEBUG] >>>> Removing outliers using method: None ...
2024-05-30 17:34:58 [src.outlier_detection:162] [INFO] >>>> No outlier detection method specified. Skipping outlier detection.
2024-05-30 17:34:58 [src.outlier_detection:150] [DEBUG] >>>> Removing outliers using method: None ...
2024-05-30 17:34:58 [src.outlier_detection:162] [INFO] >>>> No outlier detection method specified. Skipping outlier detection.
2024-05-30 17:34:58 [src.data_cleaning:

In [63]:
apply_data_cleaning = True
if apply_data_cleaning:
    train_data = cleaned_train
    test_data = cleaned_test

In [64]:
from scipy.signal import find_peaks

def extract_peaks_from_sensor_signal(dataframe, sensor_measure_columns_names):
    motor_ids = dataframe['UnitNumber'].unique()
    # TODO: Rename
    dataframes_motor_id = []
    for motor_id in motor_ids:
        dataframe_motor_id = pd.DataFrame(dataframe[dataframe['UnitNumber'] == motor_id])
        for sensor_measure_column_name in sensor_measure_columns_names:
            values = dataframe[dataframe['UnitNumber'] == motor_id][sensor_measure_column_name].values
            peaks = find_peaks(values)
            count_of_peaks = np.zeros(values.shape)
            count_of_peaks[peaks[0]] = 1
            count_of_peaks = np.cumsum(count_of_peaks)
            dataframe_motor_id[sensor_measure_column_name + ' Peaks'] = count_of_peaks
        dataframes_motor_id.append(dataframe_motor_id)
    return pd.concat(dataframes_motor_id)

sensor_measure_columns_names = list(filter(lambda x: x.startswith('Sensor'), train_data.keys()))
train_data_peaks = extract_peaks_from_sensor_signal(train_data, sensor_measure_columns_names)
test_data_peaks = extract_peaks_from_sensor_signal(test_data, sensor_measure_columns_names)

In [65]:
apply_peaks_generation = False
if apply_data_cleaning:
    train_data = train_data_peaks
    test_data = test_data_peaks

In [66]:
# Add RUL to train datasets
from src.rolling_window_creator import calculate_RUL

time_column = 'Cycle'
group_column = 'UnitNumber'

train_data = calculate_RUL(train_data, time_column, group_column)

In [67]:
# Padding of train_data and test_data
min_train = min([values.shape[0] for values in train_data.groupby("UnitNumber").indices.values()])
min_test = min([values.shape[0] for values in test_data.groupby("UnitNumber").indices.values()])
print(min(min_train, min_test))

padding = 0
if window_size > min(min_train, min_test):
    padding = window_size - min(min_train, min_test)
print(padding)

# Padding of train_data
for engine_number in train_data["UnitNumber"].unique():
    padding_data_frame = pd.DataFrame(np.zeros(shape=(padding, len(train_data.columns))), columns=train_data.columns)
    padding_data_frame["UnitNumber"] = engine_number
    padding_data_frame["Cycle"] = -1
    train_data = pd.concat([train_data, padding_data_frame])
    
# Padding of test_data
for engine_number in test_data["UnitNumber"].unique():
    padding_data_frame = pd.DataFrame(np.zeros(shape=(padding, len(test_data.columns))), columns=test_data.columns)
    padding_data_frame["UnitNumber"] = engine_number
    padding_data_frame["Cycle"] = -1
    test_data = pd.concat([test_data, padding_data_frame])

# Sort to assure the correct processing
train_data.sort_values(['UnitNumber', 'Cycle'], inplace=True, ascending=True)
test_data.sort_values(['UnitNumber', 'Cycle'], inplace=True, ascending=True)

min_train = min([values.shape[0] for values in train_data.groupby("UnitNumber").indices.values()])
min_test = min([values.shape[0] for values in test_data.groupby("UnitNumber").indices.values()])
print(min(min_train, min_test))

31
0
31


In [68]:
test_list = []
for engine_number in test_data["UnitNumber"].unique():
    rows_of_engine = test_data["UnitNumber"] == engine_number
    engine_dataframe = test_data[rows_of_engine].tail(window_size)
    #engine_dataframe = engine_dataframe.drop(columns=["UnitNumber", "Cycle"])
    engine_dataframe = engine_dataframe.drop(columns=["UnitNumber"])
    test_list.append(engine_dataframe.values)
    
X_test = np.array(test_list)
y_test = test_RUL_data.values

In [69]:
train, val = train_val_split_by_group(train_data, test_size=0.2, random_state=12)

2024-05-30 17:35:04 [src.utils:131] [INFO] >>>> Train set contains 80 different engines --> in total 16528
2024-05-30 17:35:04 [src.utils:132] [INFO] >>>>  Test set contains 20 different engines --> in total 4103


In [70]:
X_train, y_train = create_sliding_window(train, window_size=window_size, drop_columns=['UnitNumber', 'RUL'])
X_val, y_val = create_sliding_window(val, window_size=window_size, drop_columns=['UnitNumber', 'RUL'])

In [71]:
select_part_of_data_1 = True

size_compared_to_other_values = 2
if select_part_of_data_1:
    # Drop some samples with clip value 
    clip_value = 140
    count_train_value = []
    count_val_value = []
    for i in range(1, clip_value + 1):
        count_train_value.append(np.sum(y_train == i))
        count_val_value.append(np.sum(y_val == i))
        
    print(count_train_value)
    print(count_val_value)
    
    indices_train_clip_value = np.arange(y_train.shape[0])[y_train == clip_value]
    indices_val_clip_value = np.arange(y_val.shape[0])[y_val == clip_value]
    
    np.random.seed(63)
    np.random.shuffle(indices_train_clip_value)
    np.random.shuffle(indices_val_clip_value)
    
    median_train = np.median(count_train_value[:-1])
    median_val = np.median(count_val_value[:-1])
    
    indices_train_clip_value = indices_val_clip_value[:int(median_train) * size_compared_to_other_values]
    indices_val_clip_value = indices_val_clip_value[:int(median_val) * size_compared_to_other_values]
    
    print(indices_train_clip_value.shape)
    print(indices_val_clip_value.shape)
    
    new_indices_train = np.arange(y_train.shape[0])[y_train != clip_value]
    new_indices_val = np.arange(y_val.shape[0])[y_val != clip_value]
    new_indices_train = np.concatenate((new_indices_train, indices_train_clip_value))
    new_indices_val = np.concatenate((new_indices_val, indices_val_clip_value))
    print(new_indices_train.shape)
    print(new_indices_val.shape)
    
    np.random.shuffle(new_indices_train)
    np.random.shuffle(new_indices_val)
    
    # Get selected samples
    select_num_train_samples = 80 * 4
    select_num_val_samples = 20 * 4
    X_train = X_train[new_indices_train]
    y_train = y_train[new_indices_train]
    X_val = X_val[new_indices_val]
    y_val = y_val[new_indices_val]

select_part_of_data_2 = False
    
keep_from_motor = 20
if select_part_of_data_2:
    # Select from each UnitNumber some samples
    indicis_for_train = []
    indicis_for_val = []
    for motor_id in train_data['UnitNumber'].unique():
        if motor_id in X_train[:, 0, 0]:
            indices_with_motor_id = np.arange(X_train.shape[0])[X_train[:, 0, 0] == motor_id] 
            indicis_for_train.append(indices_with_motor_id[:keep_from_motor])
        else:
            indices_with_motor_id = np.arange(X_val.shape[0])[X_val[:, 0, 0] == motor_id] 
            indicis_for_val.append(indices_with_motor_id[:keep_from_motor])
        
    indicis_for_train = np.concatenate(indicis_for_train)
    indicis_for_val = np.concatenate(indicis_for_val)
    
    # Select new indices and remove UnitNumber
    X_train = X_train[indicis_for_train][:, :, 1:]
    y_train = y_train[indicis_for_train]
    #X_val = X_val[indicis_for_val][:, :, 1:]
    X_val = X_val[:, :, 1:]
    #y_val = y_val[indicis_for_val]

[80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 79, 79, 79, 79, 79, 79, 79, 78, 78, 76, 76, 76, 76, 76, 76, 76, 76, 76, 76, 74, 74, 74, 74, 74, 74, 73, 72, 72, 70, 70, 67, 67, 67, 67, 67, 66, 66, 65, 64, 64, 3358]
[20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 

In [72]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler

X_train_shape = X_train.shape
X_val_shape = X_val.shape
X_test_shape = X_test.shape

# Note: Do not normalize the cycle value! That is why we start with one
for i in range(1, X_train.shape[-1]):
    minmax_scaler = MinMaxScaler()
    robust_scaler = RobustScaler()
    std_scaler = StandardScaler()
    
    # Scale the data with StandardScaler
    X_train[:, :, i] = std_scaler.fit_transform(X_train[:, :, i])
    X_val[:, :, i] = std_scaler.transform(X_val[:, :, i])
    X_test[:, :, i] = std_scaler.transform(X_test[:, :, i])
    
    # Scale the data wit MinMaxScaler
    X_train[:, :, i] = minmax_scaler.fit_transform(X_train[:, :, i])
    X_val[:, :, i] = minmax_scaler.transform(X_val[:, :, i])
    X_test[:, :, i] = minmax_scaler.transform(X_test[:, :, i])
    
    # Scale the data wit RobustScaler
    X_train[:, :, i] = robust_scaler.fit_transform(X_train[:, :, i])
    X_val[:, :, i] = robust_scaler.transform(X_val[:, :, i])
    X_test[:, :, i] = robust_scaler.transform(X_test[:, :, i])

Change types of arrays and swap axes:

In [73]:
train_CNN = True

print(X_train.shape)
if train_CNN:
    X_train = np.swapaxes(X_train, 1, 2)
    #X_train = X_train[:, np.newaxis, :, :]
X_train = np.array(X_train, dtype=np.float32)
y_train = np.array(y_train, dtype=np.float32)
print(X_train.shape)

print(X_val.shape)
if train_CNN:
    X_val = np.swapaxes(X_val, 1, 2)
    #X_val = X_val[:, np.newaxis, :, :]
X_val = np.array(X_val, dtype=np.float32)
y_val = np.array(y_val, dtype=np.float32)
print(X_val.shape)

print(X_test.shape)
if train_CNN:
    X_test = np.swapaxes(X_test, 1, 2)
    #X_test = X_test[:, np.newaxis, :, :]
X_test = np.array(X_test, dtype=np.float32)
y_test = np.array(y_test, dtype=np.float32)
print(X_test.shape)

(11010, 30, 33)
(11010, 33, 30)
(2770, 30, 33)
(2770, 33, 30)
(100, 30, 33)
(100, 33, 30)


## Only CNN

Inspiration: Paper Dynamic predictive maintenance for multiple components using data-driven
probabilistic RUL prognostics: The case of turbofan engines

In [74]:
# Select hyperparameters of trainer!
checkpoint_callback = ModelCheckpoint(monitor="val_loss")
trainer = Trainer(min_epochs=1, max_epochs=150, callbacks=[checkpoint_callback], deterministic=True)
datamodule = TurbofanDatamodule(batch_size=128)
datamodule.set_train_dataset(X_train, y_train)
datamodule.set_val_dataset(X_val, y_val)
datamodule.set_predict_dataset(X_test)
datamodule.set_test_dataset(X_test, y_test[:, 0])
model = CNNModel(lr=0.001, window_size=window_size, features=33, dropout_rate=0.2)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [75]:
%%capture
# For visualization write 'tensorboard --logdir=lightning_logs/' in console

trainer.fit(model, datamodule=datamodule)


  | Name        | Type    | Params
----------------------------------------
0 | loss        | MSELoss | 0     
1 | dropout     | Dropout | 0     
2 | layer1_conv | Conv1d  | 6.6 K 
3 | layer2_conv | Conv1d  | 8.0 K 
4 | layer3_conv | Conv1d  | 8.0 K 
5 | layer4_conv | Conv1d  | 8.0 K 
6 | fc1         | Linear  | 153 K 
7 | fc2         | Linear  | 8.3 K 
8 | fc3         | Linear  | 65    
----------------------------------------
192 K     Trainable params
0         Non-trainable params
192 K     Total params
0.771     Total estimated model params size (MB)


In [76]:
trainer.validate(model, datamodule=datamodule, ckpt_path='best')

Restoring states from the checkpoint path at C:\Users\Johannes\PycharmProjects\damage-propagation-modeling\lightning_logs\version_149\checkpoints\epoch=45-step=4002.ckpt
Loaded model weights from the checkpoint at C:\Users\Johannes\PycharmProjects\damage-propagation-modeling\lightning_logs\version_149\checkpoints\epoch=45-step=4002.ckpt


Validation: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         val_loss          │    180.86964416503906     │
└───────────────────────────┴───────────────────────────┘

[{'val_loss': 180.86964416503906}]

In [77]:
pred = trainer.test(model, datamodule=datamodule, ckpt_path='best')

Restoring states from the checkpoint path at C:\Users\Johannes\PycharmProjects\damage-propagation-modeling\lightning_logs\version_149\checkpoints\epoch=45-step=4002.ckpt
Loaded model weights from the checkpoint at C:\Users\Johannes\PycharmProjects\damage-propagation-modeling\lightning_logs\version_149\checkpoints\epoch=45-step=4002.ckpt
C:\Users\Johannes\anaconda3\envs\PSDA\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │     267.6355285644531     │
└───────────────────────────┴───────────────────────────┘

In [31]:
from sklearn.metrics import root_mean_squared_error
from lightning import LightningModule
import torch 
model_from_checkpoint = LightningModule.load_from_checkpoint("C:/Users/Johannes/PycharmProjects/damage-propagation-modeling/lightning_logs/version_134/checkpoints/epoch=114-step=26105.ckpt")
# disable randomness, dropout, etc...
model_from_checkpoint.eval()
# predict with the model
pred = model_from_checkpoint(torch.tensor(X_test)).detach().numpy()
root_mean_squared_error(pred, torch.tensor(y_test))

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/Johannes/PycharmProjects/damage-propagation-modeling/lightning_logs/version_134/checkpoints/epoch=114-step=26105.ckpt'

In [78]:
pred = model(torch.tensor(X_test)).detach().numpy()
model.eval()
root_mean_squared_error(pred, torch.tensor(y_test))

20.74496

In [33]:
pred

array([ 51.862972,  67.89349 ,  26.498564,  95.374695,  79.14195 ,
        99.26876 ,  70.10647 ,  60.206066,  78.893234,  75.16445 ,
        68.97512 ,  75.68833 , 106.93179 ,  31.458738, 111.88981 ,
        70.50948 , 121.09294 , 133.38309 ,  88.324196,  93.30068 ,
        20.652811, 116.759964,  92.665634,  16.95975 , 100.89441 ,
        16.584661, 142.51463 , 109.31307 , 110.87555 ,  78.90963 ,
        85.24842 ,  45.71385 ,  86.61725 ,  63.96631 , 103.03752 ,
       113.956795, 127.93344 ,  37.64642 ,   9.226481,  43.823063,
       112.50242 ,  74.93342 , 115.161705,  78.06849 , 146.9397  ,
         9.05889 , 125.63947 ,  41.578754, 106.79327 ,  13.959208,
        63.320995, 141.03683 ,  81.06794 , 112.00667 ,  65.58863 ,
        62.390923,  57.31184 ,  38.89475 ,  36.391994,  92.64628 ,
       134.31812 ,  64.032   ,  80.11501 ,  34.75124 ,  94.39489 ,
        83.86167 ,  88.42881 , 127.99012 , 112.436226, 118.27959 ,
        51.29933 ,  80.9912  , 141.13812 , 146.47522 ,  49.284

In [34]:
y_test[:, 0]

array([ 44.,  51.,  27., 120., 101.,  99.,  71.,  55.,  55.,  66.,  77.,
       115., 115.,  31., 108.,  56., 136., 132.,  85.,  56.,  18., 119.,
        78.,   9.,  58.,  11.,  88., 144., 124.,  89.,  79.,  55.,  71.,
        65.,  87., 137., 145.,  22.,   8.,  41., 131., 115., 128.,  69.,
       111.,   7., 137.,  55., 135.,  11.,  78., 120.,  87.,  87.,  55.,
        93.,  88.,  40.,  49., 128., 129.,  58., 117.,  28., 115.,  87.,
        92., 103., 100.,  63.,  35.,  45.,  99., 117.,  45.,  27.,  86.,
        20.,  18., 133.,  15.,   6., 145., 104.,  56.,  25.,  68., 144.,
        41.,  51.,  81.,  14.,  67.,  10., 127., 113., 123.,  17.,   8.,
        28.], dtype=float32)

## Only LSTM

In [109]:
# Select hyperparameters of trainer!
checkpoint_callback = ModelCheckpoint(monitor="val_loss")
trainer = Trainer(min_epochs=1, max_epochs=150, callbacks=[checkpoint_callback])
datamodule = TurbofanDatamodule()
datamodule.set_train_dataset(X_train, y_train)
datamodule.set_val_dataset(X_val, y_val)
datamodule.set_predict_dataset(X_test)
datamodule.set_test_dataset(X_test, y_test[:, 0])
model = LSTMModel(lr=0.00001, window_size=window_size, features=25, dropout_rate=0.05)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [110]:
%%capture
trainer.fit(model, datamodule=datamodule)


  | Name        | Type    | Params
----------------------------------------
0 | loss        | MSELoss | 0     
1 | dropout     | Dropout | 0     
2 | lstm_layer1 | LSTM    | 79.4 K
3 | lstm_layer2 | LSTM    | 49.7 K
4 | lstm_layer3 | LSTM    | 5.2 K 
5 | fc1         | Linear  | 20.5 K
6 | fc2         | Linear  | 4.2 K 
7 | fc3         | Linear  | 65    
----------------------------------------
159 K     Trainable params
0         Non-trainable params
159 K     Total params
0.636     Total estimated model params size (MB)


In [112]:
pred = trainer.test(model, datamodule=datamodule, ckpt_path='best')

Restoring states from the checkpoint path at C:\Users\Johannes\PycharmProjects\damage-propagation-modeling\lightning_logs\version_128\checkpoints\epoch=20-step=9534.ckpt
Loaded model weights from the checkpoint at C:\Users\Johannes\PycharmProjects\damage-propagation-modeling\lightning_logs\version_128\checkpoints\epoch=20-step=9534.ckpt


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │     3046.253173828125     │
└───────────────────────────┴───────────────────────────┘

In [113]:
from sklearn.metrics import root_mean_squared_error
import torch 
pred = model(torch.tensor(X_test)).detach().numpy()
root_mean_squared_error(pred, torch.tensor(y_test))

54.869015